In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install gdown

In [2]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1zZC7jGTY6kPUIVdAMhmXvad_-K1dD88Z'
output = 'pytorch_model.bin'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145'
output = 'output.jsonl'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?export=download&id=1E0hqkgPgl9NA18OmFZrrvHJRdl26dbzj'
output = 'config.json'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1zZC7jGTY6kPUIVdAMhmXvad_-K1dD88Z
To: /content/pytorch_model.bin
100%|██████████| 1.13G/1.13G [00:13<00:00, 86.7MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145
To: /content/output.jsonl
100%|██████████| 167M/167M [00:01<00:00, 102MB/s]


'output.jsonl'

In [17]:
from transformers import XGLMTokenizer, XGLMForCausalLM, GenerationConfig
import torch
def load_model(base_model:str="facebook/xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    new_tokens = ['<human>:', '<bot>:']
    tokenizer.add_tokens(list(new_tokens))
    model = XGLMForCausalLM.from_pretrained('/content/model/', local_files_only=True,torch_dtype=torch.float16)
    return model,tokenizer
model,tokenizer = load_model()

In [41]:
from datasets import load_dataset,Dataset
datasets = load_dataset('json',data_files = 'output.jsonl')
datasets = Dataset.from_dict(datasets['train'][:10_000])
datasets

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Background:', '<human>:', '<bot>:'],
    num_rows: 10000
})

In [36]:
def format_prompt(prompt):
    return {'prompt':f"{prompt['Background:']} <human>: "}
# datasets = datasets['train']
datasets = datasets.map(format_prompt,remove_columns=['Background:', '<human>:', '<bot>:'])
datasets = datasets.map(lambda x:{'token':len(tokenizer.tokenize(x['prompt']))})
datasets = datasets.filter(lambda x:x['token']<255)
# datasets = datasets.map(preprocess,remove_columns=['prompt','token']) 
datasets

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'token'],
    num_rows: 6271
})

In [37]:
prompt = datasets['prompt'][0]

In [ ]:
import torch
generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
    )
device = torch.device('cuda')
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"]
input_ids = input_ids.to(device)
print(input_ids)

In [39]:
model.to(device)
with torch.no_grad():
  generation_output = model.generate(
      input_ids=input_ids,
      generation_config=generation_config,
      return_dict_in_generate=True,
      output_scores=True,
      max_new_tokens=128,
  )
s = generation_output.sequences[0]
output = tokenizer.decode(s)
print(output)

</s> Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins or one of the member proteins of that superfamily α and βtubulins polymerize into microtubules a major component of the eukaryotic cytoskeleton Microtubules function in many essential cellular processes including mitosis Tubulinbinding drugs kill cancerous cells by inhibiting microtubule dynamics which are required for DNA segregation and therefore cell division there are six members of the tubulin superfamily although not all are present in all species Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin with a mass of 42 kDa In contrast tubulin polymers microtubules tend to be much bigger than actin filaments due to their cylindrical nature Tubulin was long thought to be specific to eukaryotes More recently however several prokaryotic proteins have been shown to be related to tubulin <human>: Briefly describe Tubulin in molecular b

In [40]:
output

'</s> Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins or one of the member proteins of that superfamily α and βtubulins polymerize into microtubules a major component of the eukaryotic cytoskeleton Microtubules function in many essential cellular processes including mitosis Tubulinbinding drugs kill cancerous cells by inhibiting microtubule dynamics which are required for DNA segregation and therefore cell division there are six members of the tubulin superfamily although not all are present in all species Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin with a mass of 42 kDa In contrast tubulin polymers microtubules tend to be much bigger than actin filaments due to their cylindrical nature Tubulin was long thought to be specific to eukaryotes More recently however several prokaryotic proteins have been shown to be related to tubulin <human>: Briefly describe Tubulin in molecular 

In [30]:
prompt

' Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins or one of the member proteins of that superfamily α and βtubulins polymerize into microtubules a major component of the eukaryotic cytoskeleton Microtubules function in many essential cellular processes including mitosis Tubulinbinding drugs kill cancerous cells by inhibiting microtubule dynamics which are required for DNA segregation and therefore cell division there are six members of the tubulin superfamily although not all are present in all species Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin with a mass of 42 kDa In contrast tubulin polymers microtubules tend to be much bigger than actin filaments due to their cylindrical nature Tubulin was long thought to be specific to eukaryotes More recently however several prokaryotic proteins have been shown to be related to tubulin'